In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls /kaggle/input


Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 262 (delta 126), reused 221 (delta 85), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 1.13 MiB | 9.55 MiB/s, done.
Resolving deltas: 100% (126/126), done.
/kaggle/working/NIDS_Coursework/My_Code
unsw-nb-15


In [2]:
import os
import random
import numpy as np
import tensorflow as tf

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seed(42)

2025-08-27 12:22:41.337903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756297361.531833      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756297361.593331      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')



# --------------------------- perform preprocessing--------------

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)

# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)

# Filter out categorical one-hot encoded columns
numerical_columns = [col for col in processed_full_df 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_") or
                             col == 'label')]

target = processed_full_df['label']
features = processed_full_df.drop(columns=['label'])

(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X = features.astype("float32").values  # (257673, 95) as float32
y = target.astype("int32").values      # (257673,)


# reshape for (samples, timesteps, features=1)
X = np.expand_dims(X, axis=-1)  # (257673, 95, 1)

# Split train, val, test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")



input_shape = (X_train.shape[1], X_train.shape[2])


X_train: (206138, 95, 1)
y_train: (206138,)
X_val: (41228, 95, 1)
y_val: (41228,)
X_test: (10307, 95, 1)
y_test: (10307,)


In [5]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        numerator = tp * tn - fp * fn
        denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
        mcc = numerator / (denominator + 1e-10)
        specificity = tn / (tn + fp + 1e-10)
        balanced_acc = (recall + specificity) / 2
    else:
        mcc = 0
        balanced_acc = 0
        specificity = 0

    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc': mcc,
        'specificity': specificity,
        'balanced_accuracy': balanced_acc
    }

## Model Building

In [6]:
from tensorflow.keras import layers, models, optimizers, losses, metrics


model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
    

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_data=(X_val, y_val),
    verbose=1
)

y_pred = (model.predict(X_test) > 0.5).astype("int32")

normal_results = evaluate_metrics(y_true = y_test, y_pred = y_pred)
print(f'normal results: {normal_results}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1756297379.504588      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756297379.505389      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/20


2025-08-27 12:23:01.287071: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
I0000 00:00:1756297383.546452     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


806/806 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8010 - loss: 0.3878

2025-08-27 12:23:10.207740: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8011 - loss: 0.3877 - val_accuracy: 0.9039 - val_loss: 0.1905
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9098 - loss: 0.1854 - val_accuracy: 0.9173 - val_loss: 0.1679
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9183 - loss: 0.1701 - val_accuracy: 0.9231 - val_loss: 0.1600
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9206 - loss: 0.1646 - val_accuracy: 0.9251 - val_loss: 0.1564
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9225 - loss: 0.1608 - val_accuracy: 0.9270 - val_loss: 0.1536
Epoch 6/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9239 - loss: 0.1579 - val_accuracy: 0.9279 - val_loss: 0.1513
Epoch 7/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9253 - loss: 0.1555 - val_accuracy: 0.9281 - val_loss: 0.1497
Epoch 8/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9264 - loss: 0.1534 - val_accuracy: 0.9287 - val

2025-08-27 12:24:47.983636: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
normal results: {'f1': 0.9481302774427021, 'precision': 0.9417403025310769, 'recall': 0.9546075603461364, 'mcc': 0.85469167173334, 'specificity': 0.8954301075268576, 'balanced_accuracy': 0.925018833936497}


## Attack codes

In [7]:
import tensorflow as tf

@tf.function
def fgsm_attack_tf(model, inputs, labels, epsilon=0.05, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        predictions = model(inputs, training=False)
        loss = tf.keras.losses.binary_crossentropy(labels, predictions)
        loss = tf.reduce_mean(loss)

    gradients = tape.gradient(loss, inputs)
    signed_grad = tf.sign(gradients)
    x_adv = inputs + epsilon * signed_grad
    x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)
    return x_adv


@tf.function
def pgd_attack_tf(model, inputs, labels, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)

    for _ in tf.range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)
            loss = tf.reduce_mean(loss)

        gradients = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(gradients)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


@tf.function
def mi_fgsm_attack(model, inputs, labels, epsilon=0.05, alpha=0.01, steps=40, decay=1.0, clip_min=-1.5, clip_max=1.5):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
    labels = tf.reshape(tf.convert_to_tensor(labels, dtype=tf.float32), (-1, 1))

    x_adv = tf.identity(inputs)
    g = tf.zeros_like(inputs)

    for _ in tf.range(steps):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(labels, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        # Normalize gradient
        grad_norm = tf.reduce_mean(tf.abs(grad), axis=list(range(1, len(grad.shape))), keepdims=True)
        grad = grad / (grad_norm + 1e-8)

        g = decay * g + grad
        x_adv = x_adv + alpha * tf.sign(g)
        x_adv = tf.clip_by_value(x_adv, inputs - epsilon, inputs + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv


In [8]:
def evaluate_model(model, X, y,epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ---------------- Clean Evaluation ----------------
    print("✅ Clean Evaluation:")
    y_pred_clean = (model.predict(x_clean_tf) > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"  F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- FGSM Evaluation ----------------
    print("\nFGSM Attack Evaluation")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- PGD Evaluation ----------------
    print("\nPGD Attack Evaluation:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ---------------- MI-FGSM Evaluation ----------------
    print("\nMI-FGSM Attack Evaluation:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10   
        decay = 1.0   

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        y_pred_adv = (model.predict(x_adv_tf) > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred_adv)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f" MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results
        


In [9]:
print("CNN Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)



CNN Results: 
✅ Clean Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  F1: 0.9481, Precision: 0.9417, Recall: 0.9546, MCC: 0.85469167173334, Specificity: 0.8954301075268576, Balanced Acc: 0.925018833936497

FGSM Attack Evaluation
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.01 → F1: 0.8528, Precision: 0.8419, Recall: 0.8640, MCC: 0.5828981860283338, Specificity: 0.7126344086021313, Balanced Acc: 0.7883044519099923
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.03 → F1: 0.7007, Precision: 0.7606, Recall: 0.6496, MCC: 0.2773373014676086, Specificity: 0.6379032258064344, Balanced Acc: 0.6437580498244256
 92/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:24:53.403800: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.05 → F1: 0.5175, Precision: 0.5928, Recall: 0.4592, MCC: -0.09544739197268803, Specificity: 0.4413978494623537, Balanced Acc: 0.4503178711407715
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.07 → F1: 0.2628, Precision: 0.3220, Recall: 0.2220, MCC: -0.585983656670822, Specificity: 0.1723118279569846, Balanced Acc: 0.19713207915231953

PGD Attack Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.01 → F1: 0.8384, Precision: 0.8368, Recall: 0.8400, MCC: 0.550747204060085, Specificity: 0.7099462365591207, Balanced Acc: 0.7749670457123825
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.03 → F1: 0.5869, Precision: 0.6733, Recall: 0.5201, MCC: 0.0704524379306774, Specificity: 0.553225806451598, Balanced Acc: 0.5366705926139879
 89/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:24:59.214674: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.05 → F1: 0.0521, Precision: 0.0735, Recall: 0.0404, MCC: -0.8664922277601382, Specificity: 0.0983870967741909, Balanced Acc: 0.06938483425319535
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.0004, Precision: 0.0005, Recall: 0.0003, MCC: -0.9993691119333873, Specificity: 0.00026881720430106806, Balanced Acc: 0.0002862227815948941

MI-FGSM Attack Evaluation:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.01 → F1: 0.8386, Precision: 0.8364, Recall: 0.8407, MCC: 0.5507274270901635, Specificity: 0.7088709677419164, Balanced Acc: 0.7748089467523913
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.03 → F1: 0.5935, Precision: 0.6786, Recall: 0.5274, MCC: 0.08183761422998286, Specificity: 0.5577956989247161, Balanced Acc: 0.5425990791572115
 96/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:25:05.111936: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.05 → F1: 0.0658, Precision: 0.0912, Recall: 0.0515, MCC: -0.8565314287937115, Specificity: 0.09220430107526634, Balanced Acc: 0.0718346539534522
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.07 → F1: 0.0006, Precision: 0.0008, Recall: 0.0005, MCC: -0.9991589240215801, Specificity: 0.00026881720430106806, Balanced Acc: 0.00036212987131707414


## Adversarial Training

In [10]:
import tensorflow as tf
from tensorflow.keras import Model

class PGDAdversarialTrainerCNN(tf.keras.Model):
    def __init__(self, base_model, epsilon_range=(0.01, 0.1), alpha=0.01, num_iter=7, clip_min=-1.5, clip_max=1.5):
        super().__init__()
        self.base_model = base_model
        self.epsilon_range = epsilon_range
        self.alpha = alpha
        self.num_iter = num_iter
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()
        self.metric = tf.keras.metrics.BinaryAccuracy()

    def compile(self, optimizer, **kwargs):
        super().compile(**kwargs)
        self.optimizer = optimizer

    def pgd_attack_batch(self, x, y):
        x_adv = tf.identity(x)
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        epsilon = tf.random.uniform([], *self.epsilon_range)

        for _ in range(self.num_iter):
            with tf.GradientTape() as tape:
                tape.watch(x_adv)
                preds = self.base_model(x_adv, training=True)
                loss = self.loss_fn(y, preds)

            grad = tape.gradient(loss, x_adv)
            x_adv = x_adv + self.alpha * tf.sign(grad)
            # Project back to epsilon ball around original input
            x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
            x_adv = tf.clip_by_value(x_adv, self.clip_min, self.clip_max)

        return x_adv

    def train_step(self, data):
        x, y = data
        x_adv = self.pgd_attack_batch(x, y)

        # Combine clean and adversarial samples
        x_combined = tf.concat([x, x_adv], axis=0)
        y_combined = tf.concat([y, y], axis=0)
        y_combined = tf.expand_dims(tf.cast(y_combined, tf.float32), axis=-1)

        with tf.GradientTape() as tape:
            preds = self.base_model(x_combined, training=True)
            loss = self.loss_fn(y_combined, preds)

        grads = tape.gradient(loss, self.base_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.base_model.trainable_variables))
        self.metric.update_state(y_combined, preds)

        return {"loss": loss, "accuracy": self.metric.result()}

    def test_step(self, data):
        x, y = data
        y = tf.expand_dims(tf.cast(y, tf.float32), axis=-1)
        preds = self.base_model(x, training=False)
        loss = self.loss_fn(y, preds)
        self.metric.update_state(y, preds)
        return {"loss": loss, "accuracy": self.metric.result()}

    def call(self, inputs):
        return self.base_model(inputs)


In [11]:
# Wrap your existing CNN

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',   # monitor validation loss
    patience=3,           # stop if no improvement for 3 epochs
    restore_best_weights=True
)


adv_model = PGDAdversarialTrainerCNN(
    base_model=model,           
    epsilon_range=(0.01, 0.05),
    alpha=0.01,
    num_iter=7,
    clip_min=-1.5,
    clip_max=1.5
)

adv_model.compile(optimizer=tf.keras.optimizers.Adam())

# Train with adversarial training
adv_model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=20,
    callbacks=[early_stop] 

)

print("Advesarial training Results: ")
epsilons = [.01,.03,.05,.07]
fgsm_results, pgd_results, mifgsm_results = evaluate_model(
    model=adv_model,
    X=X_test,
    y=y_test,
    epsilon_list=epsilons,
)


Epoch 1/20
804/806 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8360 - loss: 0.2931

2025-08-27 12:25:28.829164: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


806/806 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.8361 - loss: 0.2928 - val_accuracy: 0.9193 - val_loss: 0.0540
Epoch 2/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8796 - loss: 0.2122 - val_accuracy: 0.9224 - val_loss: 0.0509
Epoch 3/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8883 - loss: 0.1971 - val_accuracy: 0.9220 - val_loss: 0.0518
Epoch 4/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8947 - loss: 0.1882 - val_accuracy: 0.9235 - val_loss: 0.0537
Epoch 5/20
806/806 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8983 - loss: 0.1837 - val_accuracy: 0.9218 - val_loss: 0.0515
Advesarial training Results: 
✅ Clean Evaluation:
 31/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   

2025-08-27 12:26:25.925145: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  F1: 0.9371, Precision: 0.9640, Recall: 0.9116, MCC: 0.8362401430493449, Specificity: 0.9397849462365339, Balanced Acc: 0.9257145468999581

FGSM Attack Evaluation
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.01 → F1: 0.9194, Precision: 0.9395, Recall: 0.9001, MCC: 0.7860307926669831, Specificity: 0.8973118279569651, Balanced Acc: 0.8987090489412881
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.03 → F1: 0.9031, Precision: 0.9177, Recall: 0.8890, MCC: 0.7396670913413304, Specificity: 0.8588709677419124, Balanced Acc: 0.8739474012840426
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.05 → F1: 0.8872, Precision: 0.8991, Recall: 0.8757, MCC: 0.695194698042402, Specificity: 0.8260752688171821, Balanced Acc: 0.8508697279261256
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.07 → F1: 0.8674, Precision: 0.8822, Recall: 0.8530, MCC: 0.6438878405360369, Specificity: 0.7983870967741721, Balanced Acc: 0.8257154855360158

PGD Attack Eval

2025-08-27 12:26:31.767631: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.01 → F1: 0.9178, Precision: 0.9375, Recall: 0.8989, MCC: 0.7815686748279156, Specificity: 0.8938172043010513, Balanced Acc: 0.8963544803955537
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.03 → F1: 0.8946, Precision: 0.9087, Recall: 0.8810, MCC: 0.716465490097438, Specificity: 0.8432795698924505, Balanced Acc: 0.862128626604036
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.05 → F1: 0.8520, Precision: 0.8582, Recall: 0.8459, MCC: 0.5951707176720954, Specificity: 0.7524193548386895, Balanced Acc: 0.799163981351332
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.7325, Precision: 0.7723, Recall: 0.6967, MCC: 0.3236355423211975, Specificity: 0.636290322580628, Balanced Acc: 0.6664827960253983

MI-FGSM Attack Evaluation:
 94/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:26:37.798718: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.01 → F1: 0.9174, Precision: 0.9373, Recall: 0.8983, MCC: 0.7805970611700731, Specificity: 0.8935483870967501, Balanced Acc: 0.8959164434345144
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.03 → F1: 0.8941, Precision: 0.9084, Recall: 0.8802, MCC: 0.7150772783317139, Specificity: 0.8427419354838483, Balanced Acc: 0.861480273951124
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.05 → F1: 0.8509, Precision: 0.8543, Recall: 0.8476, MCC: 0.5899052191896242, Specificity: 0.7440860215053563, Balanced Acc: 0.7958322926716094
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
 MI-FGSM ε=0.07 → F1: 0.7436, Precision: 0.7809, Recall: 0.7097, MCC: 0.3475277305189424, Specificity: 0.6473118279569718, Balanced Acc: 0.6785215584296777


In [12]:
# generate adversarial values for gan training

# experimental PGD code (GPU-optimized, batched)
def pgd_attack_tf_fast(model, x, y, epsilon=0.05, alpha=0.01, num_iter=10, clip_min=-1.5, clip_max=1.5):
    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.reshape(tf.cast(y, tf.float32), (-1, 1))
    x_orig = tf.identity(x)
    x_adv = tf.identity(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            logits = model(x_adv, training=False)
            loss = tf.keras.losses.binary_crossentropy(y, logits)
            loss = tf.reduce_mean(loss)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x_orig - epsilon, x_orig + epsilon)
        x_adv = tf.clip_by_value(x_adv, clip_min, clip_max)

    return x_adv

def batch_pgd_attack(model, X, y, attack_fn, batch_size=1024, **kwargs):
    adv_list = []
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        x_adv_batch = attack_fn(model, X_batch, y_batch, **kwargs)
        adv_list.append(x_adv_batch)
    return tf.concat(adv_list, axis=0)

# Generate adversarial examples in batches to leverage GPU
X_adv_train = batch_pgd_attack(model, X_train, y_train, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_train shape: {X_adv_train.shape}")

X_adv_val = batch_pgd_attack(model, X_val, y_val, pgd_attack_tf_fast, epsilon=0.05, batch_size=1024)
print(f"X_adv_val shape: {X_adv_val.shape}")


X_adv_train shape: (206138, 95, 1)
X_adv_val shape: (41228, 95, 1)


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping

# ------------------------------
# Generator CNN (1D output)
# ------------------------------
class GeneratorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1D(128, 3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        # final output: 1 channel (same as clean input)
        self.output_layer = layers.Conv1D(1, 3, padding='same', activation='linear')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        return self.output_layer(x)


# ------------------------------
# Discriminator CNN (1D input)
# ------------------------------
class DiscriminatorCNN(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = layers.Conv1D(64, 3, padding='same', activation='relu')
        self.global_pool = layers.GlobalAveragePooling1D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.global_pool(x)
        x = self.dense1(x)
        return self.output_layer(x)


# ------------------------------
# GAN Purifier CNN
# ------------------------------
class GANPurifierCNN(Model):
    def __init__(self, generator, discriminator, lambda_adv=0.5):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_adv = lambda_adv
        self.bce = tf.keras.losses.BinaryCrossentropy()
        self.l1 = tf.keras.losses.MeanAbsoluteError()
        self.d_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.g_optimizer = tf.keras.optimizers.Adam(1e-4)

    def compile(self, **kwargs):
        super().compile(**kwargs)

    def train_step(self, data):
        x_clean, x_adv = data
        with tf.GradientTape(persistent=True) as tape:
            x_fake = self.generator(x_adv, training=True)
            real_pred = self.discriminator(x_clean, training=True)
            fake_pred = self.discriminator(x_fake, training=True)

            # Discriminator loss
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)

            # Generator loss (reconstruction + adversarial)
            g_loss = self.l1(x_clean, x_fake) + \
                     self.lambda_adv * self.bce(tf.ones_like(fake_pred), fake_pred)

            # Cosine similarity
            x_clean_flat = tf.reshape(x_clean, (tf.shape(x_clean)[0], -1))
            x_fake_flat = tf.reshape(x_fake, (tf.shape(x_fake)[0], -1))
            cos_sim = -tf.reduce_mean(
                tf.keras.losses.cosine_similarity(x_clean_flat, x_fake_flat)
            )

        # Compute gradients
        d_grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        g_grads = tape.gradient(g_loss, self.generator.trainable_variables)

        # Apply gradients
        self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss,
                "reconstruction_loss": self.l1(x_clean, x_fake),
                "cosine_similarity": cos_sim}





In [18]:
# ------------------------------
# Instantiate models
# ------------------------------
generator = GeneratorCNN()
discriminator = DiscriminatorCNN()

# Build models with dummy input (1 channel)
dummy_input = tf.zeros((1, 95, 1))
generator(dummy_input)
discriminator(dummy_input)

gan_cnn = GANPurifierCNN(generator, discriminator, lambda_adv=0.5)
gan_cnn.compile()

# ------------------------------
# Prepare dataset
# ------------------------------
batch_size = 512
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_adv_train))
train_dataset = train_dataset.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

early_stop = EarlyStopping(
    monitor="cosine_similarity",
    mode="max",              # higher cosine similarity is better
    patience=10,
    min_delta=1e-4,
    restore_best_weights=True,
    verbose=1
)

# ------------------------------
# Train
# ------------------------------
gan_cnn.fit(train_dataset, epochs=100, callbacks=[early_stop])


Epoch 1/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - cosine_similarity: 0.8775 - d_loss: 1.3781 - g_loss: 0.4535 - reconstruction_loss: 0.1047
Epoch 2/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - cosine_similarity: 0.9831 - d_loss: 1.3806 - g_loss: 0.3840 - reconstruction_loss: 0.0358
Epoch 3/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - cosine_similarity: 0.9889 - d_loss: 1.3795 - g_loss: 0.3767 - reconstruction_loss: 0.0283
Epoch 4/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - cosine_similarity: 0.9913 - d_loss: 1.3780 - g_loss: 0.3736 - reconstruction_loss: 0.0248
Epoch 5/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - cosine_similarity: 0.9924 - d_loss: 1.3734 - g_loss: 0.3732 - reconstruction_loss: 0.0233
Epoch 6/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - cosine_similarity: 0.9932 - d_loss: 1.3640 - g_loss: 0.3742 - reconstruction_loss: 0.0218
Epoch 7/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - cosine_similarity: 0.9938 - d_loss: 1.3469 - g_loss: 0.3777 - recon

In [21]:
# Generate purified outputs
x_purified_val = generator(X_adv_val, training=False)

# Cast to float32 for evaluation
x_purified_val = tf.cast(x_purified_val, tf.float32)
X_val_cast = tf.cast(X_val, tf.float32)

# L1 (Mean Absolute Error)
l1_loss = tf.reduce_mean(tf.abs(X_val_cast - x_purified_val))

# L2 (Mean Squared Error)
l2_loss = tf.reduce_mean(tf.square(X_val_cast - x_purified_val))

# Normalize along sequence dimension
X_val_norm = tf.nn.l2_normalize(tf.squeeze(X_val_cast, axis=-1), axis=1)           
x_purified_norm = tf.nn.l2_normalize(tf.squeeze(x_purified_val, axis=-1), axis=1)  

# Cosine similarity
cos_sim = tf.reduce_mean(tf.reduce_sum(X_val_norm * x_purified_norm, axis=1))

print(f"🧪 Purifier Evaluation Metrics:")
print(f"✅ Cosine Similarity: {cos_sim:.6f}")
print(f"   ➤ L1 Loss (MAE): {l1_loss:.6f}")
print(f"   ➤ L2 Loss (MSE): {l2_loss:.6f}")


🧪 Purifier Evaluation Metrics:
✅ Cosine Similarity: 0.994087
   ➤ L1 Loss (MAE): 0.019915
   ➤ L2 Loss (MSE): 0.001275


## GAN+Adv Training evaluation

In [22]:
def evaluate_model(model, generator, adv_model, X, y, epsilon_list=[0.01, 0.03, 0.05, 0.07]):
    import tensorflow as tf
    import numpy as np

    X_clean = X
    x_clean_tf = tf.convert_to_tensor(X_clean, dtype=tf.float32)
    y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

    # ----------- Clean Data Evaluation (no adversarial) -----------
    print("✅ Clean Evaluation (GAN purified):")
    x_purified_clean = generator(x_clean_tf, training=False)
    y_pred_clean_prob = adv_model.predict(x_purified_clean)
    y_pred_clean = (y_pred_clean_prob > 0.5).astype(int).flatten()
    clean_metrics = evaluate_metrics(y, y_pred_clean)
    print(f"Clean → F1: {clean_metrics['f1']:.4f}, Precision: {clean_metrics['precision']:.4f}, Recall: {clean_metrics['recall']:.4f}, MCC: {clean_metrics.get('mcc', 'N/A')}, Specificity: {clean_metrics.get('specificity', 'N/A')}, Balanced Acc: {clean_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- FGSM Attack Evaluation -----------
    print("\nFGSM Attack Evaluation with GAN Purification")
    fgsm_results = []
    for eps in epsilon_list:
        x_adv = fgsm_attack_tf(model, x_clean_tf, y_tf, epsilon=eps )
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        fgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    # ----------- PGD Attack Evaluation -----------
    print("\nPGD Attack Evaluation with GAN Purification:")
    pgd_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        x_adv = pgd_attack_tf(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, num_iter=10)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        pgd_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  PGD ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

      # ----------- MI-FGSM Attack Evaluation -----------
    print("\nMI-FGSM Attack Evaluation with GAN Purification:")
    mifgsm_results = []
    for eps in epsilon_list:
        alpha = eps * 0.2
        steps = 10
        decay = 1.0

        x_adv = mi_fgsm_attack(model, x_clean_tf, y_tf, epsilon=eps, alpha=alpha, steps=steps, decay=decay)
        x_adv_tf = tf.convert_to_tensor(x_adv.numpy(), dtype=tf.float32)
        x_purified = generator(x_adv_tf, training=False)
        y_pred_prob = adv_model.predict(x_purified )
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        adv_metrics = evaluate_metrics(y, y_pred)
        mifgsm_results.append({'epsilon': eps, 'adversarial': adv_metrics})

        print(f"  MI-FGSM ε={eps:.2f} → F1: {adv_metrics['f1']:.4f}, Precision: {adv_metrics['precision']:.4f}, Recall: {adv_metrics['recall']:.4f}, MCC: {adv_metrics.get('mcc', 'N/A')}, Specificity: {adv_metrics.get('specificity', 'N/A')}, Balanced Acc: {adv_metrics.get('balanced_accuracy', 'N/A')}")

    return fgsm_results, pgd_results, mifgsm_results


In [24]:
fgsm_full, pgd_full, mi_full = evaluate_model(model= model,generator=generator,adv_model= model, X = X_test, y = y_test)


✅ Clean Evaluation (GAN purified):
 96/323 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

2025-08-27 12:45:26.730904: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Clean → F1: 0.9298, Precision: 0.9318, Recall: 0.9277, MCC: 0.8062223390551224, Specificity: 0.8798387096773957, Balanced Acc: 0.9037875801309402

FGSM Attack Evaluation with GAN Purification
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.01 → F1: 0.9261, Precision: 0.9262, Recall: 0.9261, MCC: 0.7953748202800953, Specificity: 0.869354838709654, Balanced Acc: 0.8977106666601253
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.03 → F1: 0.9161, Precision: 0.9126, Recall: 0.9197, MCC: 0.76610626082655, Specificity: 0.8440860215053536, Balanced Acc: 0.8818881602896436
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.05 → F1: 0.9018, Precision: 0.9010, Recall: 0.9025, MCC: 0.7274677480030807, Specificity: 0.8244623655913756, Balanced Acc: 0.8634988311940482
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  FGSM ε=0.07 → F1: 0.8946, Precision: 0.8748, Recall: 0.9153, MCC: 0.6972005849821395, Specificity: 0.7680107526881514, Balanced Acc: 0.8416492

2025-08-27 12:45:32.502222: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.03 → F1: 0.9165, Precision: 0.9175, Recall: 0.9154, MCC: 0.7690695250433675, Specificity: 0.8543010752687942, Balanced Acc: 0.8848702886591429
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.05 → F1: 0.9004, Precision: 0.8965, Recall: 0.9044, MCC: 0.7219142436038858, Specificity: 0.8150537634408382, Balanced Acc: 0.8597054151954457
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  PGD ε=0.07 → F1: 0.8638, Precision: 0.8632, Recall: 0.8643, MCC: 0.6220970351943982, Specificity: 0.7575268817204097, Balanced Acc: 0.8109025026485759

MI-FGSM Attack Evaluation with GAN Purification:
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.01 → F1: 0.9242, Precision: 0.9262, Recall: 0.9223, MCC: 0.7908818063132403, Specificity: 0.8698924731182561, Balanced Acc: 0.8960818066213718
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.03 → F1: 0.9146, Precision: 0.9144, Recall: 0.9148, MCC: 0.7633534660727405, Specificity: 0.8483870967741707, Balan

2025-08-27 12:45:38.420751: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.05 → F1: 0.8986, Precision: 0.8967, Recall: 0.9004, MCC: 0.7179533782623362, Specificity: 0.8163978494623436, Balanced Acc: 0.8584038738734217
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
  MI-FGSM ε=0.07 → F1: 0.8580, Precision: 0.8566, Recall: 0.8594, MCC: 0.6053669760198643, Specificity: 0.7451612903225606, Balanced Acc: 0.8022906800785417
